In [87]:
import pymeteor.pymeteor as pymeteor
from os import listdir
from tqdm import tqdm
import nltk
import json
nltk.download('punkt')
from nltk.tokenize import word_tokenize

[nltk_data] Downloading package punkt to /Users/evelsve/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [73]:
main_path = '/Users/evelsve/repos/cap'
og_dataset = 'celeb_small_res'
cap_path = f'{main_path}/data/{og_dataset}/captions'
pred_path = f'{main_path}/predictions/'

In [74]:
def get_og_captions(cap_path):
    og_dictn = dict()
    for f_name in tqdm(listdir(cap_path)):
        img_id = f_name.strip('.txt')
        if 9000 <= int(img_id) <= 9999:
            with open(f'{cap_path}/{f_name}', mode='r') as f:
                lines = f.readlines()
                og_dictn[img_id] = list()
                for caption in lines:
                    caption = caption.strip('\n')
                    caption = word_tokenize(caption)
                    caption = [token.lower() for token in caption if token != ',']
                    og_dictn[img_id].append(caption)
    return og_dictn


In [88]:
def get_pred_captions(pred_file):
    with open(pred_file) as jj:
        pred_dictn = json.loads(jj.read())
    return pred_dictn

def remove_start_end(predictions):
    for img_id, prediction in predictions.items():
        prediction.remove('<start>')
        prediction.remove('<end>')
        predictions[img_id] = prediction
    return predictions

In [76]:
def hit_by_meteor(predictions, originals):
    scores = dict()
    space = ' '
    for img_id, reference_list in tqdm(originals.items()):
        scores[img_id] = dict()
        for i, reference in enumerate(reference_list):
            candidate = predictions[img_id]
            candidate = space.join(candidate)
            reference = space.join(reference)
            meteor = pymeteor.meteor(candidate, reference)
            scores[img_id][i] = meteor
    return scores


In [77]:
def max_out_meteor(results, originals):
    max_scores = dict()
    for img_id, scores in results.items():
        max_score = max(list(scores.values()))
        max_scores[img_id] = max_score
    avg = sum(max_scores.values())/len(originals)
    return max_scores, avg

In [89]:
def pipeline(cap_path, pred_path):
    all_results = dict()
    original_captions = get_og_captions(cap_path)
    for pred_file in listdir(pred_path):
        print(pred_file)
        pred_file = 'predictions_celeb_small_res_m3_trained_on_celeb_small_res.json'
        predicted_captions = get_pred_captions(f'{pred_path}/{pred_file}')
        predicted_captions = remove_start_end(predicted_captions)
        per_caption_scores = hit_by_meteor(predicted_captions, original_captions)
        per_image_scores, one_score = max_out_meteor(per_caption_scores, original_captions)
        all_results[pred_file] = one_score
    
    with open("meteor_results.json", "w") as jj:
        json.dump(all_results, jj)
            

In [90]:
pipeline(cap_path, pred_path)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 30000/30000 [00:01<00:00, 28069.68it/s]


predictions_freakshow_m9_trained_on_antons.json


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:01<00:00, 918.13it/s]


predictions_celeb_small_res_m6_trained_on_freakshow.json


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:01<00:00, 926.36it/s]


...

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:01<00:00, 777.26it/s]


predictions_celeb_small_res_m5_trained_on_sketches.json


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:01<00:00, 924.55it/s]


predictions_celeb_small_res_m9_trained_on_antons.json


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:01<00:00, 924.00it/s]
